# 3.2: Examining `G.fst` with `openFST` 
## (in `python` with `pywrapfst`)

`openFST`는 `python`에서 [`pywrapfst`](http://www.openfst.org/twiki/bin/view/FST/PythonExtension)라는 기능을 통해 구현되어 있습니다. `pywrapfst`는 `openFST`의 기능들을 `python`에서 사용할 수 있게 해줍니다. 

우리는 또한 `utils/fst_manipulate/fst_manipulate.py`에 들어있는 custom function들도 사용할 것입니다. 

In [1]:
# because of the way `kaldi` installed `openFST` we have to add the path to the python functions here
import sys
sys.path.append("/scratch/kaldi/tools/openfst-1.6.2/lib/python2.7/site-packages")    

from utils.fst_manipulate import fst_manipulate as fstman  # scripts to further manipulate fsts

import pywrapfst as openfst  # the wrapper module
import graphviz as dot       # a wrapper for graphviz, which will allow us to visualize

## read in the `fst`

`Fst.read()` 기능을 사용하면 `fst`의 내용을 다음과 같이 읽기 쉬운 포맷으로 변경하여 출력할 수 있습니다. 

 ```
 from_state     to_state    arc_symbol    weight(-log)
 ```

In [ ]:
fst_in = openfst.Fst.read("resource_files/fst/animal_fst-2_gram.fst")
print(fst_in)

## visualize

`fst_in`을 입력하면 이 `notebook`에서는 자동으로 `FST`를 시각화하기 위한 랜더링을 진행합니다. 

In [ ]:
fst_in

## write to `.dot`

`fst` 파일을 `.dot` 포맷으로 저장하여서 이 `notebook`이 아닌 외부에서도 열어볼 수 있습니다. 

In [ ]:
fst_in.draw("resource_files/fst/animal_fst-2_gram.dot")

`.dot` 파일의 기본 세팅은 `landscape` 포맷으로 결과를 출력하는 것입니다. 

In [ ]:
%%bash
head resource_files/fst/animal_fst-2_gram.dot

이 파일은 `jupyter notebook`에서 시각화가 잘 이루어지진 않습니다. 그러므로 아래와 같은 `python` 명령어를 이용하면 `FST.draw()` 기능과 동시에 `.dot` 파일을 `orientation = Portrait`으로 변경할 수 있습니다. 

In [ ]:
fstman.write_wrapper(
    fst_=fst_in, 
    path_out="resource_files/fst/animal_fst-2_gram.dot"
)    

`.dot` 포맷을 http://www.webgraphviz.com/ 과 같은 사이트에서 따로 랜더링을 할 수 있습니다. `resource_files/fst/animal_fst-2_gram.dot` 파일의 내용을 복사하여서 사용하시면 됩니다. 

## analyzing the `fst`


다음 번에 살펴볼 [이 포스트](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)는 language model을 `FST`로 변환하는 것을 다루고 있습니다. 해당 부분은 다음과 같습니다. 

![sample_g](resource_files/fst/sample_G.png)

```One thing to keep in mind here is that the weights of the FSTs are calculated by negating the natural logarithm of the probabilities, and the quantities given in ARPA file format are base 10 logarithms of probabilities. The WFST produced by arpa2fst is really straightforward but let's look little closer. There is a start node representing the start of an utterance (node 0), separate nodes for each of our "ache", "Cay", "K." vocabulary words (nodes 6, 4 and 5 respectively), a back-off node (1) and a final node(2) for the end of an utterance. Let's for the sake of the example trace a path through the graph corresponding to a bigram - say "<s> ache" bigram. Because there is no such bigram in our toy corpus we are forced to take the route through the back-off node, i.e. the arcs 0-3, 3-1, 1-6. The weight of the first arc is 0 (i.e. 1 when converted to probability), the weight of 3-1 arc is 0.69315, which corresponds to the back-off probability for "<s>" (−ln(10−0.30103)), and the weight 2.0794 of 1-6 arc corresponds to the unigram probability of "ache" (−ln(10−0.9030899)).```

`FST` 형태의 `2-gram` language model은 다음과 같은 형태를 가집니다.
 - `start` 노드가 존재합니다
    - `start` 노드에서 `<s>`로 가는 weight는 항상 0입니다.
 - `<s>`와 `</s>`를 포함한 각각의 단어에 해당하는 노드가 존재합니다.
    - 사전에 존재하는 `2-gram`에 해당하는 노드를 연결하는 `arc`가 존재합니다.
       - `arc`의 `weight`는 $p(to|from)$에 해당하는 `2-gram` probability입니다. 
    - `backoff` 노드로 가는 `arc`도 존재합니다.
       - `arc`의 `weight`는 $p(to)$에 해당하는 `backoff` probability입니다. 
 - `backoff` 노드가 존재합니다.
    - 각각의 단어 노드에서 `backoff` 노드로 이어지는 `arc`가 존재합니다.
       - `arc`는 `<eps>`라고 이름이 붙어 있습니다. 
       - `arc`의 `weight`는 $p(from)$의 `backoff unigram` probability입니다. 

`fst_manipulate.py` 파일에는 `index_fst()`라는 메쏘드가 있습니다. 이 메쏘드는 존재하는 `FST`를 입력 받아서 두 개의 `<dict>`를 반환합니다.
 - 하나의 `<dict>`는 `state_id`와 각각의 단어에서 (1) 다른 노드에서 해당 노드로 이어지는 `arc`의 `weight`와 (2) 해당 노드에서 다른 노드로 이어지는 `arc`의 `weight`를 담고 있습니다. 
 - 다른 하나의 `<dict>`는 `node_id`를 `실제 단어`로 변환할 수 있는 사전입니다. 


In [ ]:
fst_dict, node_2_word = fstman.index_fst(
    fst_in=fst_in
)

In [ ]:
fst_dict

In [ ]:
node_2_word

예를 들어 `FST`에서 어떠한 노드가 `"tyrannosaurus"`라는 단어에 해당하는 노드인지 확인하려면 다음과 같이 할 수 있습니다. 

In [ ]:
fst_dict["tyrannosaurus"]["state_id"]

그리고 **실제 모델**에서 `2-gram`(*e.g.,* `"the tyrannosaurus"`)의 확률을 알고 싶으면 다음과 같은 명령어를 사용할 수 있습니다. 

In [ ]:
fst_dict["tyrannosaurus"]["weights_from"]["the"]

같은 값을 다음과 같은 방법으로도 얻을 수 있습니다. 

In [ ]:
fst_dict["the"]["weights_to"]["tyrannosaurus"]

`<dict>`에서 `tyannosaurus`에서 다른 단어로 이어지는 `weights_from`을 찾아보면, 다음의 두 노드로만 이어진다는 것을 알 수 있습니다. 
 - `"the"`
 - `<eps>`

In [ ]:
fst_dict["tyrannosaurus"]["weights_from"]

그 이유는, **주어진** 데이터에서 구축한 `2-gram` 모델에서 `"tyrannosaurus"`로 끝나는 경우는 `"the tyrannosaurus"`밖에 없고, `<eps>`는 데이터에 없는 경우를 다루기 위한 `backoff`이기 때문입니다. 

그렇기 때문에, 만약 `"cat tyrannosaurus"`를 검색하면 에러가 납니다. 

In [ ]:
fst_dict["tyrannosaurus"]["weights_from"]["cat"]

In [ ]:
fst_dict["cat"]["weights_to"]["tyrannousaurus"]

그러한 경우에는 `"<eps> tyrannosaurus"`와 같은 방법으로 probability를 확인할 수 있습니다. 

In [ ]:
fst_dict["tyrannosaurus"]["weights_from"]["<eps>"]

In [ ]:
fst_dict["<eps>"]["weights_to"]["tyrannosaurus"]

`fst_dict`는 나중에도 사용할 것입니다. 그 전에 먼저 `FST`를 이요하여 할 수 있는것이 무엇인지 확인해보겠습니다. 
 1. 우리가 구축한 language model에 해당 `n-gram`이 존재하는지 확인할 수 있습니다. 
 2. 주어진 `n-gram`을 `FST`에 통과시키는 가장 빠른 `path`를 찾을 수 있으며, `cost`가 얼마인지도 확인할 수 있습니다. 

### checking to see if sequence is valid according to language model

우리가 `FST`를 사용하는 이유는, `ASR` pipeline에 들어온 음성 파일을 해석할 때, 가장 그럴듯한 `transcription`을 결정하기 위해서입니다. 지금은 `FST`를 사용해서 구축한 `language model`에서 주어진 문자열을 처리하기 위해서 어떻게 `FST`를 통과하는지를 살펴보도록 하겠습니다. 


In [ ]:
sample_sentence = "the rex ate the human"

`fst_manipulate.py`에는 `sequence_to_fst()`라는 기능이 있습니다. 이 기능을 이용하여서 우리는 `<str>`로 대표되는 어떠한 문자열이라도 "mini"-`FST`로 변환할 수 있습니다. 

이 기능은 두 개의 논항을 필요로 합니다. 
 - `seq_string` --> training할 문자열입니다. 
 - `lm_fst` --> 구축한 language model에서 가져온 `FST`입니다. 이 `FST`는 단어와 `index`를 제대로 매칭하기 위해 필요합니다. 
 
이 기능을 이용하면 우리가 사용한 `sample_sentence`에 해당하는 기초 단계의 `FST`를 생성할 수 있습니다. 

In [ ]:
sample_sentence_fst = fstman.sequence_to_fst(
    seq_string=sample_sentence,    
    lm_fst=fst_in                  
)                                 
sample_sentence_fst

그러고 나면, 원래의 `FST`(language model에서 구축)와 새로이 생성한 "mini"-`FST`(`sample_sentence`에서 생성)을 하나로 합친([`compose`](http://www.openfst.org/twiki/bin/view/FST/ComposeDoc)) 새로운 `FST`를 얻을 수 있습니다. 

만약 우리가 사용한 `sample_sentence`가 이전에 구축한 `language model`에서 모델링을 할 수 있다면 우리는 그 결과를 나타내는 `FST`를 출력으로 갖게 됩니다. 만약 `compose` 단계에서 오류가 발생한다면, 우리는 기존의 `language model`을 이용해서 `sample_sentence`를 처리할 수 없다는 것을 알 수 있습니다. 

**Note:** language model 구축에서 `<unk>`라는 단어를 추가하였기 때문에, 기존 language model은 어떠한 문자열이라도 처리할 수 있습니다. 

`fst_manipulate.py`는 `check_sequence()`라는 메쏘드도 가지고 있습니다 해당 메쏘드는 다음 두 개의 기능을 합니다. 
 1. `sequence_to_fst()`를 불러와서 "mini"-`FST`를 구축합니다.
 2. `compose` 기능을 불러와서 새로운 `FST`를 생성합니다.

In [ ]:
sample_sentence_fst_out = fstman.check_sequence(
    seq_string=sample_sentence, 
    lm_fst=fst_in
)
sample_sentence_fst_out

이제 우리는 language model에서 `sample_sentence`를 분석하는데 필요한 부분만 가지고 와서 볼 수 있게 되었습니다. 즉, 기존의 language model을 이용하여서 주어진 `sample_sentence`를 모델링 할 수 있다는 것입니다. 

### finding the shortest path and calculating its cost

`FST`를 살펴보면 한 노드에서 **하나 이상의** 경로를 선택할 수 있다는 것을 확인할 수 있습니다. 그러므로 주어진 문장이 **어떤 경로를 통해서 해석될 지** 확인하기 위해서, 가장 **짧은** 경로의 cost(loss)를 계산하여야 합니다. 이 cost는 language model에서 나타난 확률과 결국은 **같은 개념**이지만, 그 값이 동일하지는 않습니다. 

**Note:** 위의 이유는 `language model`을 `FST`로 변환할 때 `kaldi`에서 사용하는 방법때문입니다. 하지만 결국 두 문장의 **cost**를 비교하는 것은 두 문장의 **probability**를 비교하는 것과 동일합니다. **cost**가 크다는 것은, 그 방법을 택할 **probability**가 낮다는 것을 의미합니다. 

`fstmanipulate.py` 파일에는 `get_shortest_path()`라는 메쏘드가 있습니다. 이 메쏘드는 `FST`를 통해서 parsing 가능한 최단 경로를 반환합니다. 

In [ ]:
fstman.get_shortest_path(
    fst_in=sample_sentence_fst_out
)

이제 우리는 각 arc의 cost를 이용하여 전체 경로의 cost를 결정하여야 합니다. 결과값은 `FST`를 기반으로 한 문장의 `likelihood` 값을 나타냅니다. 

`fstmanipulate.py` 파일에는 그를 위한 `calculate_cost()` 메쏘드가 있습니다. 


In [ ]:
fstman.calculate_cost(
    fst_in=sample_sentence_fst_out
)

결과값은 `negative log, base e` (-ln$) 입니다. 그러므로 이를 로그에서 probability 값으로 변환해야 합니다. 

`fst_manipulate.py` 파일에는 이를 위한 `convert_neg_log_e()`라는 메쏘드가 있습니다. 

In [ ]:
fstman.convert_neg_log_e(
    neg_log_e=fstman.calculate_cost(
        fst_in=sample_sentence_fst_out
    )
)

## Comparing likelihoods


만약 이 `FST`가 language model을 정확하게 나타낸다면, `1.4. Examining language models`에서 비교하였던 값들이 여기서도 똑같이 비교 가능하여야 합니다. 

### `mouse ate` v. `lion ate`


이전에 두 문자열을 비교하였을 때, `"lion ate"`가 `"mouse ate"`보다 **네 배** 높은 확률을 가졌었습니다. 우리는 아래의 문장들을 이용하여 language model을 구축하였기 때문에, 이는 당연하다고 볼 수 있습니다. 

In [ ]:
cat resource_files/language_model/animal_corpus.txt

`FST`를 이용한 값을 비교하였을 때에도 같은 현상이 유지되는지 확인하도록 하겠습니다. 

In [ ]:
mouse_ate = "mouse ate"
mouse_ate_fst = fstman.check_sequence(        # generates the composed-FST of our sequence and the full FST
    seq_string=mouse_ate,    
    lm_fst=fst_in                  
)
mouse_ate_log_cost = fstman.calculate_cost(   # get the log cost
    fst_in=mouse_ate_fst
)      
mouse_ate_cost = fstman.convert_neg_log_e(    # convert to probability
    neg_log_e=mouse_ate_log_cost
)  
mouse_ate_cost

In [ ]:
lion_ate = "lion ate"
lion_ate_fst = fstman.check_sequence(         # generates the composed-FST of our sequence and the full FST
    seq_string=lion_ate,    
    lm_fst=fst_in                  
)
lion_ate_log_cost = fstman.calculate_cost(    # get the log cost
    fst_in=lion_ate_fst
)         
lion_ate_cost = fstman.convert_neg_log_e(     # convert to probability
    neg_log_e=lion_ate_log_cost
)    
lion_ate_cost

In [ ]:
lion_ate_cost / mouse_ate_cost

지금까지의 과정에서 `kaldi`가 값들을 자주 바꿔왔기 때문에, 이 정도면 결과값이 유사하다고 볼 수 있습니다. 

### `ate the mouse` v. `ate the lion`

`1.4. Examining language models`를 다시 살펴보면, `2-gram` language model은 위의 두 문장을 제대로 모델링하지 못했던 것을 볼 수 있습니다. 

주어진 문장에서 보았을 때, `"ate the mouse"`는 세 번 등장했고 `"ate the lion"`은 한 번 등장했기 때문에, `"ate the mouse"`가 등장할 확률이 **세 배** 높을 것이라고 예측할 수 있습니다. 하지만 이는 사실이 아니었습니다. 그 이유는 `"the lion"`이라는 `2-gram`이 `"the mouse"`보다 **한 번** 더 등장했기 때문입니다. 

(*c.f.,* `3-gram` language model에서는 결과값이 주어진 두 문장을 제대로 모델링 합니다.)

In [ ]:
cat resource_files/language_model/animal_corpus.txt

`FST`를 통해 값을 살펴보아도, 이전과 마찬가지의 결과를 보여줍니다. 

In [ ]:
ate_the_mouse = "ate the mouse"
ate_the_mouse_fst = fstman.check_sequence(        # generates the composed-FST of our sequence and the full FST
    seq_string=ate_the_mouse,    
    lm_fst=fst_in                  
)
ate_the_mouse_log_cost = fstman.calculate_cost(   # get the log cost
    fst_in=ate_the_mouse_fst
)      
ate_the_mouse_cost = fstman.convert_neg_log_e(    # convert to probability
    neg_log_e=ate_the_mouse_log_cost
)  
ate_the_mouse_cost

In [ ]:
ate_the_lion = "ate the lion"
ate_the_lion_fst = fstman.check_sequence(         # generates the composed-FST of our sequence and the full FST
    seq_string=ate_the_lion,    
    lm_fst=fst_in                  
)
ate_the_lion_log_cost = fstman.calculate_cost(    # get the log cost
    fst_in=ate_the_lion_fst
)      
ate_the_lion_cost = fstman.convert_neg_log_e(     # convert to probability
    neg_log_e=ate_the_lion_log_cost
)  
ate_the_lion_cost

## converting a `3-gram` language model to `FST`

`2-gram`을 만들 때와 같은 방법으로 `3-gram` language model을 `FST`로 변환할 수 있습니다. 하지만 모델의 복잡성은 급격하게 증가합니다. `resource_files/fst`에서 `3-gram` language model의 `FST`를 확인할 수 있습니다. 

In [ ]:
fst_3_gram = openfst.Fst.read("resource_files/fst/animal_fst-3_gram.fst")
fst_3_gram

여기에서 우리는 전체 단어의 갯수는 `2-gram` language model과 같은 정도로 적지만, 전체 `FST`는 훨씬 크다는 것을 볼 수 있습니다. 

In [ ]:
num_states_2_gram = len(list(fst_in.states()))
num_states_3_gram = len(list(fst_3_gram.states()))
print("number of states in 2-gram model: {}\nnumber of states in 3-gram model: {}".format(
    num_states_2_gram, num_states_3_gram
    )
)